In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lead
from pyspark.sql.types import (IntegerType, LongType)
from pyspark.sql.functions import (month, year)
from pyspark.sql.functions import udf, from_unixtime, unix_timestamp
#from pyspark import SparkContext

In [ ]:

def estado_actividad(u_id, nxt_u_id, per, nxt_per):
    
    if u_id == nxt_u_id and 12*(nxt_per.year - per.year) + (nxt_per.month - per.month) >= 4:#12*(year(nxt_per) - year(per)) + (month(nxt_per) - month(per)) >= 4:
        
        return 1
    else:
        return 0


In [2]:
spark = SparkSession.builder.appName("Preprocessing").getOrCreate()

In [ ]:
raw_data_act = spark.read.parquet('activityObfmod5.parquet')


In [ ]:
raw_data_act_int = raw_data_act.withColumn('user_id', raw_data_act['user_id'].cast(LongType()))

In [ ]:
df_act_wo_duplicates = raw_data_act_int.dropDuplicates()

In [ ]:
#df_act_wo_duplicates.write.csv("act_wo_duplicates.csv", header = True)

In [ ]:
df_act_wo_duplicates = spark.read.csv("act_wo_duplicates.csv/", header = True, inferSchema = True)


In [ ]:
#df_act_ordered = df_act_wo_duplicates.orderBy(['user_id', 'periodo'])

In [ ]:
### CREO COLUMNAS next_user_id Y next_periodo

from pyspark.sql import Window

w = Window.orderBy(['user_id','periodo']) ## --> esto ordena por user_id y periodo

df_act_orderer_w_lead = df_act_wo_duplicates.withColumn("next_periodo", lead(df_act_wo_duplicates.periodo).over(w))\
            .withColumn("next_user_id", lead(df_act_wo_duplicates.user_id).over(w))

In [ ]:
## Para merge de df's de act. y sb, ver --> https://stackoverflow.com/questions/38063657/pyspark-merge-outer-join-two-data-frames

In [ ]:
define_estados = udf(estado_actividad, IntegerType())    

   

In [ ]:
df_act_final = df_act_orderer_w_lead.withColumn("estado", define_estados(df_act_orderer_w_lead.user_id,
                                                         df_act_orderer_w_lead.next_user_id,
                                                         df_act_orderer_w_lead.periodo,
                                                         df_act_orderer_w_lead.next_periodo)) 

In [ ]:
df_act_final.select(['user_id','periodo','estado']).write.csv("df_act_final.csv", header = True)

#### Dataframe act con estados

In [ ]:
df_act_final= spark.read.csv('df_act_final.csv/', header = True, inferSchema = True)

In [ ]:
from_pattern = 'yyyy-MM-dd hh:mm:ss'
to_pattern = 'yyyy-MM-dd'

df_act_final = df_act_final.withColumn("periodo", from_unixtime(unix_timestamp(df_act_final.periodo, 
            from_pattern), to_pattern))

In [ ]:
#df_act_final.select(['user_id','periodo','estado']).show()
df_act_final.select('periodo').show()

#### Dataframe sb

In [ ]:
df_sb = spark.read.parquet('sbObfmod5.parquet')

In [ ]:
#df_sb.printSchema()

In [ ]:
df_sb_int = df_sb.withColumn('user_id', df_sb['user_id'].cast(LongType()))
df_sb_int.printSchema()

In [ ]:
#df_sb_wo_duplicates = df_sb_int.drop_duplicates()

In [ ]:
df_sb_int.count()

In [ ]:
# df_sb_int.cache()
# df_act_final.cache()

#### Hago un 'join' entre los data frames de actividad y features, según user_id y período

In [ ]:
df_sb_act = df_sb_int.orderBy(['user_id', 'periodo']).join(df_act_final.orderBy(['user_id', 'periodo']),
                                                           on = ['user_id', 'periodo'], how='inner' )

In [ ]:
#df_sb_act.count()

In [ ]:
#df_sb_act.filter(df_sb_act.estado == 1).count()

In [ ]:
#df_sb_act.dropDuplicates().count()

In [ ]:
#df_sb_act.dropDuplicates().filter(df_sb_act.estado == 1).count()

In [ ]:
df_sb_act.dropDuplicates().write.csv('dataFrame_actividad_features.csv', header = True)

### df_sb_act es el dataframe con los features de cada usuario + esado de actividad (1--> se fuga, 0--> no se fuga). 
### Después de sacarse de encima las filas duplicadas, la cantidad filas con estado = 1 es 23.665, mientras que la cantidad total de filas es 6.897.273


### ---------------------------------------------------------------------------------

In [12]:
df_sb_act = spark.read.csv('dataFrame_actividad_features.csv/', header = True, inferSchema = True)

In [17]:
df_sb_act.columns

['user_id',
 'periodo',
 'fecha_proceso',
 'R04PERREF',
 'total_deudores_informados',
 'deuda_vigente',
 'deuda_directa_morosa90',
 'deuda_directa_vencida',
 'deuda_directa_mora180',
 'deuda_indirecta_mora180',
 'deuda_directa_inversiones_financieras',
 'deuda_directa_ops_pacto',
 'deuda_indirecta_vigente',
 'deuda_indirecta_vencida',
 'deuda_directa_comercial',
 'deuda_directa_cred_consumo',
 'deuda_directa_hipotecaria',
 'deuda_directa_comercial_ext',
 'deuda_indirecta_comercial_ext',
 'deuda_directa_leasing',
 'deuda_morosa_leasing',
 'monto_lineas_cred_disp',
 'estado']

In [66]:
df_sb_act_ordered = df_sb_act.orderBy(['estado', 'periodo'])

In [95]:
def define_estado_mes_anterior(estado, next_estado, next_2_estado, next_3_estado, 
                               user_id, next_user_id, next_2user_id, next_3user_id):
    
    if (user_id == next_user_id) and (estado == 0 and next_estado == 1):
        return 1
    elif (user_id == next_2user_id) and (estado == 0 and next_2_estado == 1):
        return 1
    elif (user_id == next_3user_id) and (estado == 0 and next_3_estado == 1):
        return 1    
    elif estado == 1:  #
        return 0
    else:
        return 0
    

In [89]:

from pyspark.sql import Window

w = Window.orderBy(['user_id','periodo']) ## --> esto ordena por user_id y periodo

            

In [96]:
#    '''Para cada usuario: si estado = 1, asigna 1 al estado en los tres períodos (meses) anteriores '''

define_estados_2 = udf(define_estado_mes_anterior, IntegerType())    

df_sb_act_final = df_sb_act_ordered.withColumn('estado', define_estados_2(df_sb_act_ordered.estado,
                                                           lead(df_sb_act_ordered.estado,1).over(w),
                                                           lead(df_sb_act_ordered.estado,2).over(w),
                                                           lead(df_sb_act_ordered.estado,3).over(w),
                                                                          df_sb_act_ordered.user_id,
                                                           lead(df_sb_act_ordered.user_id,1).over(w),
                                                           lead(df_sb_act_ordered.user_id,2).over(w),
                                                           lead(df_sb_act_ordered.user_id,3).over(w)))

In [97]:
df_sb_act_final.filter('estado = 1').count()

38146

In [65]:
df_sb_act.filter('estado=1').count()*2

47330

In [98]:
df_target_and_features = df_sb_act_final.select(['estado',
                                             'deuda_vigente',
                                             'deuda_directa_morosa90',
                                             'deuda_directa_vencida',
                                             'deuda_directa_mora180',
                                             'deuda_indirecta_mora180',
                                             'deuda_directa_inversiones_financieras',
                                             'deuda_directa_ops_pacto',
                                             'deuda_indirecta_vigente',
                                             'deuda_indirecta_vencida',
                                             'deuda_directa_comercial',
                                             'deuda_directa_cred_consumo',
                                             'deuda_directa_hipotecaria',
                                             'deuda_directa_comercial_ext',
                                             'deuda_indirecta_comercial_ext',
                                             'deuda_directa_leasing',
                                             'deuda_morosa_leasing',
                                             'monto_lineas_cred_disp',])

In [74]:
df_target_and_features.filter('estado=1').count()

49105

In [94]:
df_sb_act_final.filter('estado=1').select(['user_id','estado','periodo']).show(100)

+----------------+------+-------------------+
|         user_id|estado|            periodo|
+----------------+------+-------------------+
| 564873096719398|     1|2016-01-01 00:00:00|
| 564873096719398|     1|2016-02-01 00:00:00|
| 564873096719398|     1|2016-03-01 00:00:00|
| 564873096719398|     1|2016-04-01 00:00:00|
| 785830050894577|     1|2016-07-01 00:00:00|
| 785830050894577|     1|2016-08-01 00:00:00|
| 785830050894577|     1|2016-09-01 00:00:00|
| 785830050894577|     1|2016-10-01 00:00:00|
|1053623958002368|     1|2018-04-01 00:00:00|
|1053623958002368|     1|2018-05-01 00:00:00|
|1053623958002368|     1|2018-06-01 00:00:00|
|1053623958002368|     1|2018-07-01 00:00:00|
|1158743260739798|     1|2016-06-01 00:00:00|
|1158743260739798|     1|2016-07-01 00:00:00|
|1158743260739798|     1|2016-08-01 00:00:00|
|1158743260739798|     1|2016-09-01 00:00:00|
|1256313568775196|     1|2016-07-01 00:00:00|
|1256313568775196|     1|2016-08-01 00:00:00|
|1256313568775196|     1|2016-09-0

In [100]:
df_target_and_features.write.csv('dataFrame target and features(fuga 3 meses antes no-current).csv', header = True)

### ---------------------------------------------------------------------------------

### ---------------------------------------------------------------------------------

### cosas de prueba

In [21]:
df_aux1 = spark.createDataFrame([(1, 15, 1), (2,30,0), (3,43,1), (4,21,0), (5,344,0)], ['col1','col2', 'col3'])
df_aux2 = spark.createDataFrame([(1, 15), (2,30), (3,43), (4,21), (7,89)], ['col1','col2'])



In [ ]:
df_aux1.select(['col1','col2']).union(df_aux2).show()

In [22]:
df_aux1.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|  15|   1|
|   2|  30|   0|
|   3|  43|   1|
|   4|  21|   0|
|   5| 344|   0|
+----+----+----+



In [44]:
df_aux1.withColumn('next_2_col2', lead(df_aux1.col2,2).over(Window.orderBy('col2'))).show()

+----+----+----+-----------+
|col1|col2|col3|next_2_col2|
+----+----+----+-----------+
|   1|  15|   1|         30|
|   4|  21|   0|         43|
|   2|  30|   0|        344|
|   3|  43|   1|       null|
|   5| 344|   0|       null|
+----+----+----+-----------+



In [ ]:
df_aux2.show()

In [ ]:
new_df = df_aux2.join(df_aux1, on=['col1', 'col2'], how='inner')


In [ ]:
new_df.show()